In [1]:
from ultralytics import YOLO


model = YOLO("yolo11n-seg.pt")


In [6]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture("tesst.avi")
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    results = model(frame, imgsz=640, verbose=False)
    result = results[0]
    
    # Tạo một ảnh đen có cùng kích thước với frame gốc
    segmented_frame = np.zeros_like(frame)
    
    # Check if masks exist
    if result.masks is not None and result.boxes is not None:
        # Lấy tất cả các vùng đã được segment
        for i, mask in enumerate(result.masks.data):
            cls_idx = int(result.boxes.cls[i])
            # Chỉ lấy người hoặc tất cả các đối tượng (tùy bạn chọn)
            if result.names[cls_idx] == "person":  # Chỉ lấy người
            # if True:  # Hoặc lấy tất cả các đối tượng được segment
                mask_np = mask.cpu().numpy()
                # Resize mask to match frame shape
                mask_resized = cv.resize(mask_np, (frame.shape[1], frame.shape[0]), interpolation=cv.INTER_NEAREST)
                # Copy pixel từ frame gốc vào vùng có mask
                segmented_frame[mask_resized > 0.5] = frame[mask_resized > 0.5]
    
    cv.imshow('frame', segmented_frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


In [1]:
# Script để tổ chức data theo pattern fi* và no*
import os
import shutil
import glob

def organize_fight_data():
    """
    Tổ chức data từ folder data/ theo pattern:
    - fi*_xvid.avi -> fight/
    - no*_xvid.avi hoặc các file khác -> normal/
    """
    
    # Đường dẫn
    source_dir = "../../data"  # Từ src/data/ lên root/data/
    target_dir = "../../data_organized"
    
    # Tạo thư mục organized
    os.makedirs(target_dir, exist_ok=True)
    os.makedirs(os.path.join(target_dir, "fight"), exist_ok=True)
    os.makedirs(os.path.join(target_dir, "normal"), exist_ok=True)
    
    # Tìm tất cả video files
    video_files = glob.glob(os.path.join(source_dir, "*.avi"))
    
    print(f"Tìm thấy {len(video_files)} video files")
    
    fight_count = 0
    normal_count = 0
    
    for video_file in video_files:
        filename = os.path.basename(video_file)
        
        if filename.startswith("fi") and "_xvid.avi" in filename:
            # Fight videos
            target_path = os.path.join(target_dir, "fight", filename)
            shutil.copy2(video_file, target_path)
            fight_count += 1
        else:
            # Normal videos (no* hoặc các file khác)
            target_path = os.path.join(target_dir, "normal", filename)
            shutil.copy2(video_file, target_path)
            normal_count += 1
    
    print(f"Đã tổ chức:")
    print(f"  - {fight_count} fight videos vào data_organized/fight/")
    print(f"  - {normal_count} normal videos vào data_organized/normal/")
    
    return target_dir

# Chạy function
organized_dir = organize_fight_data()
print(f"\nData đã được tổ chức tại: {organized_dir}")
print("Bạn có thể sử dụng đường dẫn này làm args.data_preprocessed_dir khi train")

Tìm thấy 1000 video files
Đã tổ chức:
  - 500 fight videos vào data_organized/fight/
  - 500 normal videos vào data_organized/normal/

Data đã được tổ chức tại: ../../data_organized
Bạn có thể sử dụng đường dẫn này làm args.data_preprocessed_dir khi train


In [2]:
# Test SegmentationDataset
import sys
sys.path.append('..')  # Để import được từ src/

from datasets import SegmentationDataset
import torch
from torch.utils.data import DataLoader

# Tạo dataset
print("Đang khởi tạo SegmentationDataset...")
dataset = SegmentationDataset(
    video_dir="../../data_organized",
    classes_list=['fight', 'normal'],
    model_path="../../yolo11n-seg.pt",
    target_class="person",
    sequence_length=10,  # Giảm xuống để test nhanh
    image_height=224,
    image_width=224,
)

print(f"Dataset size: {len(dataset)}")
print(f"Classes: {dataset.classes_list}")

# Test load một sample
print("\nĐang test load 1 sample...")
frames, label = dataset[0]
print(f"Frames shape: {frames.shape}")  # Should be (T, C, H, W)
print(f"Label: {label}")
print(f"Label type: {type(label)}")

# Test với DataLoader
print("\nĐang test với DataLoader...")
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)

for batch_idx, (batch_frames, batch_labels) in enumerate(dataloader):
    print(f"Batch {batch_idx}:")
    print(f"  Batch frames shape: {batch_frames.shape}")  # Should be (B, T, C, H, W)
    print(f"  Batch labels shape: {batch_labels.shape}")  # Should be (B,)
    print(f"  Labels: {batch_labels}")
    break  # Chỉ test 1 batch

print("\nSegmentationDataset hoạt động bình thường! ✅")

Đang khởi tạo SegmentationDataset...
Classes detected: ['fight', 'normal']
Dataset size: 1000
Classes: ['fight', 'normal']

Đang test load 1 sample...
Frames shape: torch.Size([10, 3, 224, 224])
Label: 0
Label type: <class 'torch.Tensor'>

Đang test với DataLoader...
Batch 0:
  Batch frames shape: torch.Size([2, 10, 3, 224, 224])
  Batch labels shape: torch.Size([2])
  Labels: tensor([0, 0])

SegmentationDataset hoạt động bình thường! ✅


In [3]:
# Script để chạy training với segmentation data
import os

# Tạo script training
training_script = """
# Script để chạy training với dữ liệu đã được tổ chức
import sys
import os
sys.path.append('src')

# Import cần thiết
from src.configs.configs import parse_arguments
from src.train_3dcnn import main

# Tạo args giả lập để test
class Args:
    def __init__(self):
        self.data_preprocessed_dir = "data_organized"
        self.num_classes = 2
        self.sequence_length = 30
        self.image_height = 224
        self.image_width = 224
        self.img_channels = 3
        self.hidden_size = 128
        self.dropout_prob = 0.3
        self.learning_rate = 0.001
        self.optimizer = "AdamW"
        self.batch_size = 4  # Giảm batch size cho test
        self.epochs = 2  # Ít epochs để test
        self.val_split = 0.2
        self.seed = 42
        self.model_save_path = "weights/segmented_model.pth"
        
        # Learning rate scheduler
        self.lr_reduce_monitor = "val_loss"
        self.lr_reduce_factor = 0.5
        self.lr_reduce_patience = 3
        self.lr_reduce_min_lr = 1e-6
        self.lr_reduce_verbose = True
        
        # Early stopping (disabled for test)
        self.es_patience = 10
        self.es_verbose = True
        self.es_delta = 0.001
        self.es_monitor = "val_loss"
        self.es_restore_best_weights = True

if __name__ == "__main__":
    # Kiểm tra data_organized có tồn tại không
    if not os.path.exists("data_organized"):
        print("ERROR: Chưa có thư mục data_organized!")
        print("Hãy chạy cell tổ chức data trước")
        exit()
    
    # Tạo mock args
    import argparse
    args = Args()
    
    print("Bắt đầu training với segmentation data...")
    print(f"Data directory: {args.data_preprocessed_dir}")
    print(f"Số classes: {args.num_classes}")
    print(f"Sequence length: {args.sequence_length}")
    print(f"Batch size: {args.batch_size}")
    print(f"Epochs: {args.epochs}")
    
    # Chạy training
    main(args)
"""

# Ghi script ra file
with open("../../train_segmentation.py", "w", encoding="utf-8") as f:
    f.write(training_script)

print("Đã tạo script training: train_segmentation.py")
print("\nĐể chạy training:")
print("1. cd d:\\code\\FightDetection")
print("2. python train_segmentation.py")
print("\nHoặc chạy từ terminal trong VS Code!")

Đã tạo script training: train_segmentation.py

Để chạy training:
1. cd d:\code\FightDetection
2. python train_segmentation.py

Hoặc chạy từ terminal trong VS Code!
